## tesing dataloader V5

new ideas with ordinal c and yc 
has ordinal binary and continous outputs

In [1]:
# Load dependencies
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split



from utils.configuration import *
from utils.loss_ordinal import *
from utils.tram_model_helpers import *
from utils.tram_models import *
from utils.tram_data import *


import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

Train with GPU support.


adjustet funcitnos for ordinal outcomes 

dev ordinal

In [2]:
experiment_name = "testing_v6_dataloader"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

In [3]:

# datatype= "ordinal_Xc_Yc" # as X its continous aswell as Y continous
# datatype= "ordinal_Xc_Yo"
# datatype= "ordinal_Xn_Yc"
# datatype= "ordinal_Xn_Yo"


In [3]:
df = pd.DataFrame({
    "x1": np.random.normal(loc=0, scale=1, size=1000),
    "ord_bin1":  np.random.binomial(1, p=0.4, size=1000),
    "ord_multi": np.random.choice([0, 1, 2, 3], size=1000, p=[0.2, 0.3, 0.3, 0.2]),
    "ord_bin2":  np.random.binomial(1, p=0.4, size=1000),
})

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]

In [6]:
df = pd.DataFrame({
    "ord_multi": np.random.choice([0, 1, 2, 3], size=1000, p=[0.2, 0.3, 0.3, 0.2]),
    "x1": np.random.normal(loc=0, scale=1, size=1000),
})

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]


data_type={key:value for key, value in zip(train_df.columns, ['ordinal_Xc_Yc']+['continous'])}
print(data_type)
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

adj_matrix=np.array([['0', 'ls'],
                     ['0', '0']])
return_intercept_shift=True

nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)
print(levels_dict)

for node in target_nodes:
    print(f'-----------------{node}-------------------')
    train_loader, _ = get_dataloader(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=return_intercept_shift, verbose=False)
    
    if return_intercept_shift:
        (int_input, shift_list), y =next(iter(train_loader))
        print(f'int_input {int_input}')
        print(f'shift_list {shift_list}')

        print(int_input.shape)

{'ord_multi': 'ordinal_Xc_Yc', 'x1': 'continous'}
{'ord_multi': 4}
-----------------ord_multi-------------------


int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------x1-------------------
int_input tensor([[1.]])
shift_list [tensor([[3.]])]
torch.Size([1, 1])


In [7]:
df = pd.DataFrame({
    "ord_multi": np.random.choice([0, 0.25, 0.5, 0.75], size=1000, p=[0.2, 0.3, 0.3, 0.2]),
    "x1": np.random.normal(loc=0, scale=1, size=1000),
})

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]


data_type={key:value for key, value in zip(train_df.columns, ['ordinal_Xc_Yc']+['continous'])}
print(data_type)
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

adj_matrix=np.array([['0', 'ls'],
                     ['0', '0']])
return_intercept_shift=True

nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)
print(levels_dict)

for node in target_nodes:
    print(f'-----------------{node}-------------------')
    train_loader, _ = get_dataloader(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=return_intercept_shift, verbose=False)
    
    if return_intercept_shift:
        (int_input, shift_list), y =next(iter(train_loader))
        print(f'int_input {int_input}')
        print(f'shift_list {shift_list}')

        print(int_input.shape)

{'ord_multi': 'ordinal_Xc_Yc', 'x1': 'continous'}
{'ord_multi': 4}
-----------------ord_multi-------------------
int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------x1-------------------


int_input tensor([[1.]])
shift_list [tensor([[0.7500]])]
torch.Size([1, 1])


In [ ]:
# data_type={key:value for key, value in zip(train_df.columns, ['continous']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo'])}
# print(data_type)
# configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

# adj_matrix=np.array([['0', '0', 'cs11', 'cs11'],
#                      ['0', '0', 'cs12', 'cs12'],
#                      ['0', '0', '0', 'cs'],
#                      ['0', '0', '0', '0']])

# return_intercept_shift=True

# nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
# levels_dict=create_levels_dict(df,data_type)
# target_nodes=create_node_dict(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)

# for node in target_nodes:
#     print(f'-----------------{node}-------------------')
#     train_loader, _ = get_dataloader(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=return_intercept_shift, verbose=False)
    
#     if return_intercept_shift:
#         (int_input, shift_list), y =next(iter(train_loader))
#         print(f'int_input {int_input}')
#         print(f'shift_list {shift_list}')

#         print(int_input.shape)



{'x1': 'continous', 'ord_bin1': 'ordinal_Xn_Yo', 'ord_multi': 'ordinal_Xn_Yo', 'ord_bin2': 'ordinal_Xn_Yo'}
*************
 Model has Complex intercepts and Complex shifts, please add your Model to the modelzoo 
*************
-----------------x1-------------------
int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------ord_bin1-------------------
int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------ord_multi-------------------
int_input tensor([[1.]])
shift_list [tensor([[-0.8895,  0.0000,  1.0000]])]
torch.Size([1, 1])
-----------------ord_bin2-------------------
int_input tensor([[1.]])
shift_list [tensor([[0., 0., 1., 0.]]), tensor([[-1.2370,  0.0000,  1.0000]])]
torch.Size([1, 1])
